<img src='https://i.imgur.com/RDAD11M.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 1 / SPRINT 2 / NOTE 2*

# 📝 Assignment

# 카이제곱검정


## 1. 공공데이터 조작 연습. 

국가 통계 포털에서 아래 이미지를 참조하여

<https://kosis.kr/statisticsList/statisticsListIndex.do?menuId=M_01_01&vwcd=MT_ZTITLE&parmTabId=M_01_01#SelectStatsBoxDiv>

**2020년 8월**에 해당하는 규모별 미분양현황 에 대한 데이터셋을 생성하세요. 

<img src='https://i.imgur.com/aZc4UJO.png' width = 500>

- `60m이하`, `60~85m`, `85m초과`의 3개 규모와 (column)
- `서울`, `대전`, `대구`, `부산` 4개의 지역을 포함해야합니다. (row)
- `민간부문`만 포함합니다
- 데이터가 없는 경우는 0으로 처리하세요.

이후 데이터셋을 colab으로 불러오세요. 이때 변수의 이름은 `df`를 사용합니다.


In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chisquare
from scipy import stats

In [2]:
col = ['~60m2', '60~85m2', '85m2~']
row = ['서울', '대전', '대구', '부산']

data_url = 'https://docs.google.com/uc?export=download&id=1DcnkxhnK9RcL6HxSJBXgzfT3q7u6YyeA'

df = pd.read_csv(data_url, encoding='cp949')
df = df.rename(columns={df.columns[0] : '지역', df.columns[2] : '규모'})
df

,지역,부문(1),규모,2020. 08
0,서울,민간부문,소계,56
1,서울,민간부문,60㎡이하,54
2,서울,민간부문,60∼85㎡,2
3,서울,민간부문,85㎡초과,0
4,부산,민간부문,소계,1454
5,부산,민간부문,60㎡이하,577
6,부산,민간부문,60∼85㎡,735
7,부산,민간부문,85㎡초과,142
8,대구,민간부문,소계,1624
9,대구,민간부문,60㎡이하,143


In [3]:
# 데이터가 tidy 형태이므로 이를 wide 형태로 바꿔준다
df = df.pivot_table(index = df.columns[0], columns = df.columns[2], values = '2020. 08')

# 형태 변경 후 인덱스와 컬럼을 설정한다
df = df.reindex(index=row, columns=['60㎡이하',	'60∼85㎡',	'85㎡초과', '소계'])
df = df.rename(columns={df.columns[0] : col[0], df.columns[1] : col[1], df.columns[2] : col[2]})
df

규모,~60m2,60~85m2,85m2~,소계
지역,,,,
서울,54,2,0,56
대전,782,1,0,783
대구,143,1437,44,1624
부산,577,735,142,1454


## 2. 지역에 대해서 one-sample chi-square test를 실행, 해당 결과를 `chi1`에 저장 후 설명해보세요.

예시) 만약 **9월달 데이터**를 기준으로 한다면
```python
 [52+2+0, 590+665+142, 113+1061+42, 772+1+0]
```
을 비교 하게 될 것입니다.




In [4]:
chi1 = chisquare(df['소계'])
chi1

# one sample Chi-squared test의 귀무가설은 "표본집단의 분포가 기대와 동일하다."이다.
# 이에 대한 대립가설은 "표본집단의 분포가 기대와 동일하지 않다."이다.
# 그리고 신뢰도가 95%라는 전제하에 
# p-value가 0.05보다 작으면 귀무가설을 기각하고 대립가설을 채택하며, 
# 0.05보다 크면 귀무가설을 기각할 수 없다.

# one-sample chi-square test는 대상 데이터의 분포가 예상된 분포와 같은지 아닌지를 판단함
# 검정 결과 p > 0.05 라면 이 데이터의 분포는 예상 분포와 동일하다고 볼 수 있고(귀무가설 기각 불가),
# p <= 0.05 라면 이 데이터의 분포는 예상 분포와 동일하지 않다고 판단한다(귀무가설 기각).

# chisquare() 함수의 경우 매개변수로 f_exp(예상 분포값)을 따로 넣지 않으면
# 함수 내부적으로 들어온 데이터 값의 평균을 예상 분포값으로 설정하고서 결과를 보여준다.

Power_divergenceResult(statistic=1564.4572376818994, pvalue=0.0)

## 3. 지역과 규모에 대해서 two-sample chi-square test를 실행, 해당 결과를 `chi2`에 저장 후 설명해보세요.

예시) **9월달 데이터**를 기준으로 한다면

| |-60 | 60-85 | 85- |
|:-:|:-:|:-:|:-:|
|서울|52|2|0|
|대전|772|1|0|
|대구|113|1061|42|
|부산|590|665|142|

에 대해서 검정해야 할 겁니다.




In [5]:
chi2 = stats.chi2_contingency(df.drop(['소계'], axis=1))
chi2

# chi2_contingency() 함수 결과값 형식 :
# (chi2, p, dof, expected)

# two sample Chi-squared test는 두 표본집단의 분포가 동일한지 확인할 때 사용한다.
# 즉, 두 표본집단이 연관이 있는지, 없는지를 확인할 수 있다.

# H0 : 지역과 규모는 연관이 없다
# H1 : 지역과 규모는 연관이 있다
# p-value가 0.05보다 작기 때문에 귀무가설을 기각하고 대립가설을 채택한다.
# 따라서 지역과 규모는 연관이 있다.

# 이처럼 two sample Chi-squared test의 귀무가설은 "두 변수는 연관이 없다(두 변수는 독립이다)."이다.
# 이에 대한 대립가설은 "두 변수는 연관이 있다(두 변수는 독립이 아니다)."이다.
# 그리고 신뢰도가 95%라는 전제하에 p-value가 0.05보다 작으면 귀무가설을 기각하고 대립가설을 채택하며, 
# 0.05보다 크면 귀무가설을 기각할 수 없다.

(2064.5767314171994, 0.0, 6, array([[ 22.24559612,  31.09522594,   2.65917794],
        [311.04110288, 434.77789124,  37.18100587],
        [645.12228746, 901.76155221,  77.11616033],
        [577.59101353, 807.36533061,  69.04365586]]))

### 4. 2번에 대해서 NumPy 를 사용하여 (Scipy를 사용하지 않고) $\chi^2$ test 시행 후 2번의 결과와 비교해보세요. 

- `obs`, `exp`, `chi`라는 변수를 사용해야합니다.

In [6]:
obs = df['소계']

exp_val = sum(obs) / len(obs)

exp = []
for i in range(len(obs)):
  exp.append(exp_val)

squared = np.power(obs - exp, 2)

chi = np.sum(squared / exp)
p = 1 - stats.chi2.cdf(chi, len(obs)-1)

print(f'chisquare : {chi}, pvalue : {p}')

chisquare : 1564.4572376818994, pvalue : 0.0


## 🔥 도전 과제

아래 세가지를 다하면 됩니다.

### 1. Function

4번에서 사용한 one sample chisquare test를 함수의 형태로 변경하세요.

In [7]:
v1 = [18,22,20,15,23,22]
v2 = [5,23,26,19,24,23]

def myChisq(value):
  value = pd.Series(value)

  exp_val = sum(value) / len(value)

  exp = []
  for i in range(len(value)):
    exp.append(exp_val)

  squared = np.power(value - exp, 2)

  x2 = np.sum(squared / exp)
  p = 1 - stats.chi2.cdf(x2, len(value)-1)
  print(f'chisquare : {x2}, pvalue : {p}')

myChisq(v1)
myChisq(v2)

chisquare : 2.3000000000000003, pvalue : 0.8062668698851285
chisquare : 14.8, pvalue : 0.011251979028327308


### 2. ANOVA

아래 링크를 참조하여 ANOVA 에 대한 글을 읽고

<https://partrita.github.io/posts/ANOVA-python/>

다음 `4개 그룹에 대해서 평균의 차이가 있는지`에 대한 가설 검정을 시행하세요.

A : `38 33 35 92 76 97 88 41 11  9`

B : `18 52 62 48 30 40 87 12 97 82`

C :  `28 90  5 49 66 73 96 80  4 17`

D : ` 8 99  4 12  7 64 18 10  9 20`



In [8]:
a = [38, 33, 35, 92, 76, 97, 88, 41, 11, 9]
b = [18, 52, 62, 48, 30, 40, 87, 12, 97, 82]
c = [28, 90, 5, 49, 66, 73, 96, 80, 4, 17]
d = [8, 99, 4, 12, 7, 64, 18, 10, 9, 20]

In [9]:
# Null hypothesis: Groups means are equal (no variation in means of groups) 
# H0: μ1=μ2=…=μp
# Alternative hypothesis: At least, one group mean is different from other groups
# H1: All μ are not equal

stats.f_oneway(a, b, c, d)
# pvalue > 0.05 이므로 4개 그룹의 평균 차이는 없을 수도 있다.

F_onewayResult(statistic=1.7249594239128412, pvalue=0.17920877113948797)